Colab accompanying the guest lecture with the same name
in the context of  François Fleuret's [https://fleuret.org/dlc](Deep Learning Course) (EE-559, EPFL).

##### Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Import statements

In [1]:
import tensorflow as tf

print(tf.__version__)

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

1.8.0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12010668232550805097]

In [2]:
#@title show_graph(graph_def) -- run this !

# Let's visualize our graph!
# Tip: to make your graph more readable you can add a
# name="..." parameter to the individual Ops.

# src: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/tutorials/deepdream/deepdream.ipynb

import numpy as np
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# The Graph

In [0]:
x = tf.constant(1.0, name="x")
add = tf.add(x, 2.0, name="add")
mul = tf.multiply(add, 3.0,
                  name="mul")

In [0]:
show_graph(tf.get_default_graph())

## Naming, Placement, Scope

In [0]:
tf.reset_default_graph()

c = tf.constant(0, name="c")
c = tf.constant(2, name="c")
with tf.device("/device:GPU:0"):
  
  with tf.name_scope("sum"):
    s1 = tf.constant(0, name="s1")
    
    with tf.name_scope("prod"):
      p1 = tf.constant(1, name="p1")
      p2 = tf.constant(2, name="p2")
      p12 = tf.multiply(p1, p2, 
                        name="mult")

    sp = tf.add(s1, p12, name="add")

stack = tf.stack([sp, c],
                 name="stack")


In [0]:
show_graph(tf.get_default_graph())

## Conversion, Overlaoding, Broadcasting

In [0]:
tf.reset_default_graph()

a = tf.constant([1, 1])
c = a + 2

## The Graph - Inspection

In [0]:
graph = tf.get_default_graph()

In [0]:
graph.get_operations()

[<tf.Operation 'Const' type=Const>,
 <tf.Operation 'add/y' type=Const>,
 <tf.Operation 'add' type=Add>]

In [0]:
graph.get_operation_by_name('Const')

<tf.Operation 'Const' type=Const>

In [0]:
graph.get_tensor_by_name('Const:0')

<tf.Tensor 'Const:0' shape=(2,) dtype=int32>

In [0]:
show_graph(graph)

# The Session

## The Session - Makes the Tensorss Flow

In [0]:
tf.reset_default_graph()

x = tf.constant(1.0, name="x")
add = tf.add(x, 2.0, name="add")
mul = tf.multiply(add, 3.0, name="mul")

with tf.Session() as sess:
  mul, = sess.run([mul])
  print(mul)

9.0


## (Default) Graph, (Interactive) Session

In [0]:
tf.reset_default_graph()
x = tf.constant(3.1415)
sess = tf.InteractiveSession()
print(x.eval())

3.1415


In [0]:
graph = tf.Graph()
with graph.as_default():
  x = tf.constant(2.718)
with tf.Session(graph=graph) as sess:
  print(x.eval())

2.718


In [0]:
with tf.Graph().as_default(),\
      tf.Session() as sess:
  x = tf.constant(1.414)
  print(sess.run([x]))

[1.414]


## Placeholders, Feeding

In [0]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [0]:
x = tf.placeholder(tf.float32, shape=[2], name="x")
norm = tf.norm(x)

In [0]:
norm.eval(feed_dict={x: [3, 4]})

5.0

In [0]:
sess.run([x, norm], feed_dict={"x:0": [3, 4]})

[array([3., 4.], dtype=float32), 5.0]

In [0]:
sess.run([x, norm])

InvalidArgumentError: ignored

In [0]:
sess.run([x, norm], feed_dict={x: [3, 4], norm: 0})

## Graph/Session Quiz

In [0]:
with tf.Graph().as_default() as graph:
  x = tf.constant(1)
  x2 = tf.multiply(x, x)

with tf.Session() as sess:
  print(sess.run(x2))

In [0]:
with tf.Graph().as_default() as graph:
  x = tf.constant(1)
  x2 = tf.multiply(x, x)

with tf.Session(graph=graph) as sess:
  print(sess.run(x2))

In [0]:
with tf.Graph().as_default():
  x = tf.constant(1)
  x2 = tf.multiply(x, x)

  with tf.Session() as sess:
    print(sess.run(x2))

# The Shapes

In [0]:
x = tf.random_normal(shape=(2, 2))
x.shape

In [0]:
x.shape.as_list()

In [0]:
tf.reshape(x, (1, -1)).shape

## Partially Known Shapes

In [0]:
x = tf.placeholder(tf.float32, [None, 4])
x.shape

In [0]:
x2 = tf.reshape(x, (2, -1))
x2.eval({x: [range(4)]})

In [0]:
batch_size = tf.shape(x)[0]
x2b = tf.reshape(x, [batch_size, 2, -1])
x2b.eval({x: [range(4), range(0, 8, 2)]})

## Sparse Tensors

In [0]:
sparse = tf.SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4])
sparse.eval()

In [0]:
tf.sparse_tensor_to_dense(sparse).eval()

# Variables

In [0]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [0]:
v = tf.Variable(1, name="v")
print(v.eval())

In [0]:
init_op = tf.global_variables_initializer()
init_op.run()
print(v.eval())

## "v++"

In [0]:
with tf.Graph().as_default() as graph, tf.Session() as sess:
  v = tf.Variable(0, name="v")
  tf.global_variables_initializer().run()
  v = v + 1

  show_graph(graph)

In [0]:
with tf.Graph().as_default() as graph, tf.Session() as sess:
  v = tf.Variable(0, name="v")
  tf.global_variables_initializer().run()
  sess.run([tf.assign(v, v.eval() + 1)])
  sess.run([tf.assign(v, v.eval() + 1)])

  show_graph(graph)

In [0]:
with tf.Graph().as_default() as graph, tf.Session() as sess:
  v = tf.Variable(0, name="v")
  tf.global_variables_initializer().run()
  inc_v = tf.assign_add(v, 1)
  sess.run([inc_v])
  sess.run([inc_v])

  show_graph(graph)

## Naming, Sharing

In [0]:
tf.reset_default_graph()

In [0]:
v1 = tf.Variable(0, name="v1")
v2 = tf.get_variable("v2", initializer=0)
v3 = tf.get_variable("v3", shape=(3, 3), initializer=tf.random_normal_initializer())

In [0]:
v1 = tf.Variable(0, name="v1")

In [0]:
show_graph(tf.get_default_graph())

In [0]:
v2 = tf.get_variable("v2", initializer=0)

In [0]:
with tf.variable_scope("vars"):
  x = tf.get_variable("x", initializer=0.)
  
with tf.variable_scope("vars", reuse=True):
  x = tf.get_variable("x", initializer=0.)

## Sharing Example

In [0]:
tf.reset_default_graph()

In [0]:
def component(x, n, initializer=tf.random_normal_initializer):
  with tf.variable_scope("component", reuse=tf.AUTO_REUSE):
    weights = tf.get_variable("weights", shape=(n, n), initializer=initializer)
    biases = tf.get_variable("biases", shape=(n,), initializer=initializer)
    return tf.matmul(weights, x) + biases

with tf.name_scope("tower0"):
    x = tf.random_normal([5, 1])
    y0 = component(x, n=5)

with tf.name_scope("tower1"):
    x = tf.random_normal([5, 1])
    y1 = component(x, n=5)

y = y0 + y1

In [0]:
show_graph(tf.get_default_graph())

## Collections

In [0]:
tf.reset_default_graph()

In [0]:
x = tf.get_variable("x", (), tf.float32)
y = tf.get_variable("y", (), tf.float32, collections=())
z = tf.get_variable("z", (), tf.float32, collections=(), trainable=False)

In [0]:
graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[]

In [0]:
graph.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[]

## Save / Restore State

In [0]:
with tf.Graph().as_default() as graph, tf.Session() as sess:
  v = tf.get_variable("v", initializer=0.)
  #...
  tf.global_variables_initializer().run()
  saver = tf.train.Saver()
  save_path = saver.save(sess, "/tmp/model.ckpt")

In [0]:
! ls -lh /tmp/model*

-rw-r--r-- 1 root root    4 May 18 07:27 /tmp/model.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root  117 May 18 07:27 /tmp/model.ckpt.index
-rw-r--r-- 1 root root 2.3K May 18 07:27 /tmp/model.ckpt.meta


In [0]:
with tf.Session(graph=graph) as sess:
  saver.restore(sess, "/tmp/model.ckpt")
  print(v.eval())

INFO:tensorflow:Restoring parameters from /tmp/model.ckpt
0.0


In [0]:
from tensorflow.python.tools import inspect_checkpoint as chkp
chkp.print_tensors_in_checkpoint_file(
    "/tmp/model.ckpt", tensor_name='', all_tensors=True, all_tensor_names=True)

tensor_name:  v
0.0


# Putting It Together

In [3]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

1.8.0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16369082467751043331]

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

mnist.train.next_batch(2)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]]))

In [7]:
graph = tf.Graph()

with graph.as_default():
  x = tf.placeholder(tf.float32, [None, 784])
  y_ = tf.placeholder(tf.float32, [None, 10])

  W = tf.Variable(tf.zeros([784, 10]))
  b = tf.Variable(tf.zeros([10]))

  logits = tf.matmul(x, W) + b
  y = tf.nn.softmax(logits)

  cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), axis=1))
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.5)
  # Note: This call is commented here and moved two cells down in order to show
  # the graph before and after the call.
#   train_step = optimizer.minimize(cross_entropy)

In [6]:
show_graph(graph)

In [11]:
with graph.as_default():
#   train_step = optimizer.minimize(cross_entropy)

'GradientDescent'

In [10]:
show_graph(graph)

In [0]:
with tf.Session(graph=graph) as sess:
  sess.run(tf.global_variables_initializer())

  # train
  for _ in range(1100):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

  # evaluate
  correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9188


## tf.keras

NOTE : You might have to restart the kernel at this point.

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential([
    Dense(10, activation='softmax', input_dim=784)])

model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(mnist.train.images, mnist.train.labels, epochs=2)
model.evaluate(mnist.test.images, mnist.test.labels)

Using TensorFlow backend.


Epoch 1/2
55000/55000 [==============================] - 3s 47us/step - loss: 0.8029 - acc: 0.8083
Epoch 2/2
10000/10000 [==============================] - 0s 29us/step


[0.4040625471949577, 0.8945]

## Eager

NOTE: You have to restart the kernel at this point.

In [0]:
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
W = tf.contrib.eager.Variable(tf.zeros([784, 10]))
b = tf.contrib.eager.Variable(tf.zeros([10]))

def get_preds(x):
  return tf.nn.softmax(tf.matmul(x, W) + b)

for _ in range(1100):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  with tf.contrib.eager.GradientTape() as tape:
    preds = get_preds(batch_xs)
    loss = tf.reduce_mean(-tf.reduce_sum(batch_ys * tf.log(preds), axis=1))
  dW, db = tape.gradient(loss, [W, b])
  W.assign_sub(dW * 0.5)
  b.assign_sub(db * 0.5)

accuracy = lambda x,y: (x.argmax(axis=1) == y.argmax(axis=1)).mean()
accuracy(get_preds(mnist.test.images).numpy(),  mnist.test.labels)

0.9179